In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

In [5]:
raw_data_benign = load_data("Power_System_Data/cyberphysical_Benign.csv")
raw_data_benign.head()

,frame.time,eth.src,eth.dst,ip.src,ip.dst,_ws.col.Protocol,ip.len,tcp.srcport,tcp.dstport,udp.srcport,udp.dstport,PLCID,V1,I1,Theta,P,Q,1_P,1_Q,2_P,2_Q,3_P,3_Q,4_P,4_Q,5_P,5_Q,6_P,6_Q,7_P,7_Q,9_P,9_Q,10_P,10_Q,11_P,11_Q,12_P,12_Q,13_P,13_Q,14_P,14_Q,BreakerStatus_1,BreakerStatus_10,BreakerStatus_11,BreakerStatus_12,BreakerStatus_13,BreakerStatus_14,BreakerStatus_2,BreakerStatus_3,BreakerStatus_4,BreakerStatus_5,BreakerStatus_7,BreakerStatus_9,BreakerStatus_GEN,BreakerStatus_LOAD
0,"Sep 10, 2024 15:45:37.334723000 CDT",02:42:c0:a8:06:14,02:42:c0:a8:06:0a,192.168.6.20,192.168.6.10,Modbus/TCP,64.0,47582.0,502.0,NaN,NaN,11,"65,530",20,0.537,"-3,512,882","-1,800,000",-,-,-,-,-,-,-,-,-,-,"585,608,000","796,694,272",-,-,-,-,"-589,120,896","-798,494,272",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,"Sep 10, 2024 15:45:37.335540000 CDT",02:42:c0:a8:06:0a,02:42:c0:a8:06:14,192.168.6.10,192.168.6.20,Modbus/TCP,65.0,502.0,47582.0,NaN,NaN,11,"65,530",20,0.537,"-3,512,882","-1,800,000",-,-,-,-,-,-,-,-,-,-,"585,608,000","796,694,272",-,-,-,-,"-589,120,896","-798,494,272",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,"Sep 10, 2024 15:45:37.335554000 CDT",02:42:c0:a8:06:14,02:42:c0:a8:06:0a,192.168.6.20,192.168.6.10,TCP,52.0,47582.0,502.0,NaN,NaN,11,"65,530",20,0.537,"-3,512,882","-1,800,000",-,-,-,-,-,-,-,-,-,-,"585,608,000","796,694,272",-,-,-,-,"-589,120,896","-798,494,272",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,"Sep 10, 2024 15:45:37.335714000 CDT",02:42:c0:a8:06:14,02:42:c0:a8:06:0a,192.168.6.20,192.168.6.10,Modbus/TCP,64.0,47582.0,502.0,NaN,NaN,11,"65,530",20,0.537,"-3,512,882","-1,800,000",-,-,-,-,-,-,-,-,-,-,"585,608,000","796,694,272",-,-,-,-,"-589,120,896","-798,494,272",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,"Sep 10, 2024 15:45:37.336032000 CDT",02:42:c0:a8:06:0a,02:42:c0:a8:06:14,192.168.6.10,192.168.6.20,Modbus/TCP,65.0,502.0,47582.0,NaN,NaN,11,"65,530",20,0.537,"-3,512,882","-1,800,000",-,-,-,-,-,-,-,-,-,-,"585,608,000","796,694,272",-,-,-,-,"-589,120,896","-798,494,272",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [6]:
raw_data_FDI = load_data("Power_System_Data/cyberphysical_FDI_labeled.csv")
raw_data_FDI.head()

,frame.time,eth.src,eth.dst,ip.src,ip.dst,_ws.col.Protocol,ip.len,tcp.srcport,tcp.dstport,udp.srcport,udp.dstport,PLCID,V1,I1,Theta,P,Q,1_P,1_Q,2_P,2_Q,3_P,3_Q,4_P,4_Q,5_P,5_Q,6_P,6_Q,7_P,7_Q,9_P,9_Q,10_P,10_Q,11_P,11_Q,12_P,12_Q,13_P,13_Q,14_P,14_Q,BreakerStatus_1,BreakerStatus_10,BreakerStatus_11,BreakerStatus_12,BreakerStatus_13,BreakerStatus_14,BreakerStatus_2,BreakerStatus_3,BreakerStatus_4,BreakerStatus_5,BreakerStatus_7,BreakerStatus_9,BreakerStatus_GEN,BreakerStatus_LOAD,label
0,"Sep 10, 2024 17:17:30.655099000 CDT",02:42:c0:a8:03:14,02:42:c0:a8:03:0a,192.168.3.20,192.168.3.10,Modbus/TCP,64.0,41442.0,502.0,NaN,NaN,12,"65,440",32,1.288,"-6,112,586","-1,599,928",-,-,-,-,-,-,-,-,-,-,"212,578,176","349,222,304",-,-,-,-,-,-,-,-,-,-,"-218,690,000","-350,820,416",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
1,"Sep 10, 2024 17:17:30.655394000 CDT",02:42:c0:a8:03:0a,02:42:c0:a8:03:14,192.168.3.10,192.168.3.20,Modbus/TCP,61.0,502.0,41442.0,NaN,NaN,12,"65,440",32,1.288,"-6,112,586","-1,599,928",-,-,-,-,-,-,-,-,-,-,"212,578,176","349,222,304",-,-,-,-,-,-,-,-,-,-,"-218,690,000","-350,820,416",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
2,"Sep 10, 2024 17:17:30.656029000 CDT",02:42:c0:a8:03:14,02:42:c0:a8:03:03,192.168.3.20,192.168.23.4,TLSv1.2,760.0,36314.0,9200.0,NaN,NaN,12,"65,440",32,1.288,"-6,112,586","-1,599,928",-,-,-,-,-,-,-,-,-,-,"212,578,176","349,222,304",-,-,-,-,-,-,-,-,-,-,"-218,690,000","-350,820,416",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
3,"Sep 10, 2024 17:17:30.656047000 CDT",02:42:0a:01:03:04,02:42:0a:01:03:03,10.1.3.4,192.168.23.4,TLSv1.2,760.0,36314.0,9200.0,NaN,NaN,12,"65,440",32,1.288,"-6,112,586","-1,599,928",-,-,-,-,-,-,-,-,-,-,"212,578,176","349,222,304",-,-,-,-,-,-,-,-,-,-,"-218,690,000","-350,820,416",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
4,"Sep 10, 2024 17:17:30.660085000 CDT",02:42:0a:01:03:03,02:42:0a:01:03:04,192.168.23.4,10.1.3.4,TLSv1.2,495.0,9200.0,36314.0,NaN,NaN,12,"65,440",32,1.288,"-6,112,586","-1,599,928",-,-,-,-,-,-,-,-,-,-,"212,578,176","349,222,304",-,-,-,-,-,-,-,-,-,-,"-218,690,000","-350,820,416",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0


In [7]:
raw_data_ransom = load_data("Power_System_Data/cyberphysical_Ransom_labeled.csv")
raw_data_ransom.head()

,frame.time,eth.src,eth.dst,ip.src,ip.dst,_ws.col.Protocol,ip.len,tcp.srcport,tcp.dstport,udp.srcport,udp.dstport,PLCID,V1,I1,Theta,P,Q,1_P,1_Q,2_P,2_Q,3_P,3_Q,4_P,4_Q,5_P,5_Q,6_P,6_Q,7_P,7_Q,9_P,9_Q,10_P,10_Q,11_P,11_Q,12_P,12_Q,13_P,13_Q,14_P,14_Q,BreakerStatus_1,BreakerStatus_10,BreakerStatus_11,BreakerStatus_12,BreakerStatus_13,BreakerStatus_14,BreakerStatus_2,BreakerStatus_3,BreakerStatus_4,BreakerStatus_5,BreakerStatus_7,BreakerStatus_9,BreakerStatus_GEN,BreakerStatus_LOAD,label
0,"Sep 11, 2024 15:41:52.026066000 CDT",02:42:c0:a8:04:14,02:42:c0:a8:04:0a,192.168.4.20,192.168.4.10,Modbus/TCP,64.0,46010.0,502.0,NaN,NaN,13,"65,503",74,2.352,"-13,512,875","-5,800,001",-,-,-,-,-,-,-,-,-,-,"218,723,728","230,035,152",-,-,-,-,-,-,-,-,"-663,491,712","-876,173,696",-,-,"431,255,104","640,338,560",-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
1,"Sep 11, 2024 15:41:52.026559000 CDT",02:42:c0:a8:04:0a,02:42:c0:a8:04:14,192.168.4.10,192.168.4.20,Modbus/TCP,61.0,502.0,46010.0,NaN,NaN,13,"65,503",74,2.352,"-13,512,875","-5,800,001",-,-,-,-,-,-,-,-,-,-,"218,723,728","230,035,152",-,-,-,-,-,-,-,-,"-663,491,712","-876,173,696",-,-,"431,255,104","640,338,560",-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
2,"Sep 11, 2024 15:41:52.027691000 CDT",02:42:c0:a8:04:14,02:42:c0:a8:04:03,192.168.4.20,192.168.23.4,TLSv1.2,794.0,53566.0,9200.0,NaN,NaN,13,"65,503",74,2.352,"-13,512,875","-5,800,001",-,-,-,-,-,-,-,-,-,-,"218,723,728","230,035,152",-,-,-,-,-,-,-,-,"-663,491,712","-876,173,696",-,-,"431,255,104","640,338,560",-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
3,"Sep 11, 2024 15:41:52.027707000 CDT",02:42:0a:01:04:04,02:42:0a:01:04:03,10.1.4.4,192.168.23.4,TLSv1.2,794.0,53566.0,9200.0,NaN,NaN,13,"65,503",74,2.352,"-13,512,875","-5,800,001",-,-,-,-,-,-,-,-,-,-,"218,723,728","230,035,152",-,-,-,-,-,-,-,-,"-663,491,712","-876,173,696",-,-,"431,255,104","640,338,560",-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
4,"Sep 11, 2024 15:41:52.031188000 CDT",02:42:0a:01:04:03,02:42:0a:01:04:04,192.168.23.4,10.1.4.4,TLSv1.2,495.0,9200.0,53566.0,NaN,NaN,13,"65,503",74,2.352,"-13,512,875","-5,800,001",-,-,-,-,-,-,-,-,-,-,"218,723,728","230,035,152",-,-,-,-,-,-,-,-,"-663,491,712","-876,173,696",-,-,"431,255,104","640,338,560",-,-,-,-,-,-,-,-,-,-,-,-,-,-,0
